In [1]:
import os
from pathlib import Path
import librosa
import pandas as pd
import numpy as np
import shutil

In [10]:
def get_audio_files_path():
    lab2_tmp_path = Path(__file__).parent.parent / "lab2" / "tmp"
    lab2_tmp_path.mkdir(parents=True, exist_ok=True)
    lab2_audio_files_path = lab2_tmp_path / "audio_files"
    lab2_audio_files_path.mkdir(parents=True, exist_ok=True)
    audio_files_path = Path(__file__).parent.parent / "lab1" / "tmp" / "useful_audios"
    destination_path = Path(__file__).parent.parent / "lab2" / "tmp" / "audio_files"
    for file in audio_files_path.iterdir():
        if file.is_file():
            new_file_path = destination_path / file.name
            shutil.copy(file, new_file_path)

    return destination_path

audio_files_path = Path(os.getcwd()) / "tmp" / "audio_files"

In [11]:
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sr).T, axis=0)
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T, axis=0)
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr).T, axis=0)
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y).T, axis=0)
    harmonic = np.mean(librosa.effects.harmonic(y).T, axis=0)
    percussive = np.mean(librosa.effects.percussive(y).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr).T, axis=0)
    return [
        mfccs,
        chroma,
        spectral_contrast,
        tonnetz,
        spectral_centroid,
        spectral_rolloff,
        zero_crossing_rate,
        harmonic,
        percussive,
        mel,
    ]

In [12]:
def process_files(dir_path: Path):
    # Initialize empty lists to hold the features and labels
    features, file_names = [], []

    # Iterate over all files
    for file in dir_path.iterdir():

        feature = np.hstack(extract_features(file))

        features.append(feature)
        file_names.append(file.name)

    return features, file_names

features, file_names = process_files(audio_files_path)

In [13]:
df = pd.DataFrame(features)
df["Arquivo"] = file_names

# Make 'Arquivo' the first column
df = df.set_index("Arquivo").reset_index()

print(df.head())

      Arquivo           0           1          2          3          4  \
0  a00001.mp3 -357.475494  144.278671  25.227043 -12.069102 -12.000586   
1  a00002.mp3 -362.867981  113.480652  55.047539  22.268412   5.548545   
2  a00003.mp3 -374.741455  128.402176  23.937555 -14.066549 -12.737453   
3  a00004.mp3 -369.259033  116.922211  34.424026   4.118651  -2.764186   
4  a00005.mp3 -378.585327  138.650772  18.954666 -19.314791 -16.075703   

          5         6          7          8  ...       188       189  \
0 -5.494565 -7.416517  -8.400575 -12.329534  ...  0.000052  0.000049   
1  3.088625  2.680373   1.093082  -2.414867  ...  0.000090  0.000085   
2 -6.483592 -8.334821  -7.998796 -11.647849  ...  0.000041  0.000042   
3  2.279199  1.071293   5.531829  -2.866853  ...  0.000059  0.000053   
4 -9.724780 -9.246498 -10.084473 -12.905710  ...  0.000039  0.000038   

        190       191       192       193       194       195       196  \
0  0.000046  0.000050  0.000058  0.000053  0.00

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif

In [19]:
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="mean")),  # Fill missing values
        ("scaler", StandardScaler()),  # Standardize features
        # (
        #     "selector",
        #     SelectKBest(score_func=f_classif, k=10),
        # ),  # Select the 10 best features
    ]
)

X = df[df.columns[1:]]
y = df['Arquivo']

# Apply the pipeline
df[df.columns[1:]] = pipeline.fit_transform(X, y)
print(df.head())

      Arquivo         0         1         2         3         4         5  \
0  a00001.mp3 -1.000608  0.807643 -0.514552 -1.656485 -2.003320 -2.329342   
1  a00002.mp3 -1.104156 -0.454005  1.646190  1.407403  0.706870 -0.362043   
2  a00003.mp3 -1.332155  0.157259 -0.607986 -1.834715 -2.117118 -2.556030   
3  a00004.mp3 -1.226880 -0.313021  0.151845 -0.212075 -0.576902 -0.547567   
4  a00005.mp3 -1.405967  0.577095 -0.969038 -2.303008 -2.632659 -3.298922   

          6         7         8  ...       188       189       190       191  \
0 -2.536283 -2.510274 -2.855687  ... -0.137103 -0.136634 -0.132527 -0.141612   
1  0.748761 -0.291258 -0.005078  ... -0.130871 -0.130028 -0.125722 -0.135218   
2 -2.835056 -2.416364 -2.659693  ... -0.138922 -0.137921 -0.132899 -0.143141   
3  0.225243  0.746241 -0.135030  ... -0.135914 -0.135877 -0.130590 -0.140891   
4 -3.131672 -2.903863 -3.021347  ... -0.139370 -0.138653 -0.134904 -0.144635   

        192       193       194       195       196     

In [21]:
processed_audio_features_csv_path = (
    Path(os.getcwd()) / "tmp" / "processed_audio_features.csv"
)
# Save the processed DataFrame to a CSV file
df.to_csv(processed_audio_features_csv_path, index=False)